In [7]:
import pandas as pd
import numpy as np

days = 252

periods = 5
Nums = periods 
sim = 1
fwd = np.zeros((sim, Nums, periods))
Dev = np.zeros((sim, Nums))
Timestep = np.zeros(periods + 1)
correl = np.zeros(periods + 1)
fwdvol = np.zeros(periods + 1)
Notional = np.zeros(periods + 1)
fixed_rate = np.zeros(periods + 1)
df = np.zeros((sim, Nums, periods + 1))
MtM = np.zeros((sim, periods))
sumfxd = 0
sumflt = 0
Orgsumfxd = 0
Orgsumflt = 0


for k in range(periods + 1):
    Timestep[k] = 1
    correl[k] = 1
    fwdvol[k] = 0.2
    fixed_rate[k]= 0.01
    Notional[k] = 1000000
    
#print(Timestep)


for i in range(sim):
    count = 1
    df[i, 0, 0] = 1
    j = 0
    for k in range(periods):
        fwd[i, 0, k] = 0.02 + k * 0.001
        
#print(DF)
        
    for k in range(periods):
        df[i, j, k + 1] = np.exp(-fwd[i, j, k] * Timestep[k]) * df[i, j, k]
        Orgsumfxd += Notional[k] * fixed_rate[k] * df[i, j, k + 1]
        Orgsumflt += Notional[k] * fwd[i, j, k] * df[i, j, k + 1]

    #print(OrgSumFxd)
    #print(OrgSumFlt)

    for j in range(1, periods):
        drift = 0    
        for k in range(j, periods):
            Z1 = np.random.normal(0,1)
            drift += (Timestep[k-1] * fwd[i, j-1, k] * fwdvol[j] * correl[k-1]) / (1 + fwd[i, j-1, k] * Timestep[k-1]) 
            fwd[i, j, k] = fwd[i, j-1, k] * np.exp((drift * fwdvol[k-1] - 0.5 * fwdvol[k-1] ** 2) * Timestep[k-1] + fwdvol[k-1] * Timestep[k-1] ** 0.5 * Z1)
            df[i, j, j] = 1
            df[i, j, k + 1] = np.exp(fwd[i, j, k] * Timestep[k]) * df[i, j, k]
            
            sumfxd += Timestep[k] * Notional[k] * fixed_rate[k] * df[i, j, k + 1]
            sumflt += Timestep[k] * Notional[k] * fwd[i, j, k] * df[i, j, k + 1]
            
        MtM[i, j] = sumfxd + sumflt
    
        print(df)


[[[1.         0.98019867 0.95982913 0.93894347 0.91759423 0.89583414]
  [0.         1.         1.02297701 1.04253283 1.06753154 1.0902286 ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]]]
[[[1.         0.98019867 0.95982913 0.93894347 0.91759423 0.89583414]
  [0.         1.         1.02297701 1.04253283 1.06753154 1.0902286 ]
  [0.         0.         1.         1.01958289 1.04407643 1.07057443]
  [0.         0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.        ]]]
[[[1.         0.98019867 0.95982913 0.93894347 0.91759423 0.89583414]
  [0.         1.         1.02297701 1.04253283 1.06753154 1.0902286 ]
  [0.         0.         1.         1.01958289 1.04407643 1.07057443]
  [0.         0.         0.         1.         1.01892765 1.04054499]
  [0.         0.

In [122]:

import numpy as np

# Initialize parameters
periods = 5
nums = periods + 1
sim = 1

# Initialize arrays
fwd = np.zeros((sim, nums, periods))
df = np.zeros((sim, nums, periods + 1))
mtm = np.zeros((sim, periods))

time_step = np.ones(periods + 1)
correl = np.ones(periods + 1)
fwd_vol = np.full(periods + 1, 0.35)
fixed_rate = np.full(periods + 1, 0.01)
notional = np.full(periods + 1, 1000000)

# Simulation loop
for i in range(sim):
    df[i, 0, 0] = 1
    fwd[i, 0] = 0.02 + np.arange(periods) * 0.001
    
    # Calculate initial discount factors and original sums
    df[i, 0, 1:] = np.exp(-fwd[i, 0] * time_step[:-1]).cumprod()
    org_sum_fxd = np.sum(notional[:-1] * fixed_rate[:-1] * df[i, 0, 1:])
    org_sum_flt = np.sum(notional[:-1] * fwd[i, 0] * df[i, 0, 1:])
    
    # Loop over time periods and simulate forward rates
    for j in range(1, periods):
        drift = sum_fxd = sum_flt = 0
        for k in range(j, periods):
            z1 = np.random.normal(0, 1)
            drift += (time_step[k-1] * fwd[i, j-1, k] * fwd_vol[k] * correl[k-1]) / (1 + fwd[i, j-1, k] * time_step[k-1])
            fwd[i, j, k] = fwd[i, j-1, k] * np.exp((drift * fwd_vol[k-1] - 0.5 * fwd_vol[k-1] ** 2) * time_step[k-1] + fwd_vol[k-1] * time_step[k-1] ** 0.5 * z1)
            
            df[i, j, k] = df[i, j-1, k]
            df[i, j, k + 1] = np.exp(-fwd[i, j, k] * time_step[k]) * df[i, j, k]
            
            sum_fxd += time_step[k] * notional[k] * fixed_rate[k] * df[i, j, k + 1]
            sum_flt += time_step[k] * notional[k] * fwd[i, j, k] * df[i, j, k + 1]
        
        mtm[i, j] = sum_fxd + sum_flt
    
    print(mtm)
    print(org_sum_fxd + org_sum_flt)


[[    0.         97370.33017611 67694.6738257  49119.43053822
  18983.61009961]]
149945.82461598358


In [5]:
import numpy as np

periods = 5
sim = 3  # Number of simulations
fwd = np.zeros((sim, periods, periods))
df = np.zeros((sim, periods, periods + 1))
Timestep = np.ones(periods + 1)
fwdvol = np.full(periods + 1, 0.35)
fixed_rate = np.full(periods + 1, 0.01)
Notional = np.full(periods + 1, 1000000)

for i in range(sim):
    df[i, 0, 0] = 1
    for k in range(periods):
        fwd[i, 0, k] = 0.02 + k * 0.001

    for j in range(periods):
        df[i, j, j] = 1  # set diagonal to 1
        for k in range(j + 1, periods):
            Z1 = np.random.normal(0, 1)
            drift = (Timestep[k-1] * fwd[i, j, k] * fwdvol[j]) / (1 + fwd[i, j, k] * Timestep[k-1])
            fwd[i, j, k] = fwd[i, j-1, k] * np.exp((drift * fwdvol[k-1] - 0.5 * fwdvol[k-1] ** 2) * Timestep[k-1] + fwdvol[k-1] * Timestep[k-1] ** 0.5 * Z1)
            df[i, j, k + 1] = np.exp(-fwd[i, j, k] * Timestep[k]) * df[i, j, k]

# print results
for i in range(sim):
    print(f"Simulation {i+1}:")
    print(df[i])
    print()


Simulation 1:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

Simulation 2:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

Simulation 3:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]

